<a href="https://colab.research.google.com/github/sisifo3/P_T_3/blob/main/getLabels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

from torchvision import transforms

import torch
from torch.utils.data import Dataset
#from utils import multihot
#from image_utils import clahe, get_aug


# def getImagesLabels(filename):
#     df = pd.read_csv(filename)

#     X = df['Path']
#     y = df[['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']]

#     return np.asarray(X), np.asarray(y)

def getImagesLabels(filename, policy):
    """
    filename: path to the csv file containing all the imagepaths and associated labels
    """
    df = pd.read_csv(filename)
    relevant_cols = ['Path', 'Enlarged Cardiomediastinum', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 
                     'Pneumonia', 'Pneumothorax', 'Pleural Other', 'Support Devices', 'No Finding',
                     'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

    df = df[relevant_cols]
    df = df.replace(np.nan, 0.0)

    if policy == 'zeros':
        df = df.replace(-1.0, 0.0)
    elif policy == 'ones':
        df = df.replace(-1.0, 1.0)
    elif policy == 'both':
        df['Cardiomegaly'] = df['Cardiomegaly'].replace(-1.0, 0.0)
        df['Consolidation'] = df['Consolidation'].replace(-1.0, 0.0)

        df['Atelectasis'] = df['Atelectasis'].replace(-1.0, 1.0)
        df['Edema'] = df['Edema'].replace(-1.0, 1.0)
        df['Pleural Effusion'] = df['Pleural Effusion'].replace(-1.0, 1.0)

    elif policy == 'multi' or policy == 'ignore':
        df = df.replace(-1.0, 2.0)

    # df = df[df['Path'].str.contains('frontal')] ###

    X = df['Path']
    y = df[['Enlarged Cardiomediastinum', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 
            'Pneumonia', 'Pneumothorax', 'Pleural Other', 'Support Devices', 'No Finding',
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']]

    return np.asarray(X), np.asarray(y)


In [17]:
policy = 'zeros'
filename = '/content/drive/MyDrive/train.csv'
x,y = getImagesLabels(filename, policy)


In [ ]:
df = pd.read_csv(filename)
df.head()

In [25]:
print(len(y[10]))

16


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

from torchvision import transforms

import torch
from torch.utils.data import Dataset

def getImagesLabels(filename):

    df = pd.read_csv(filename)
    relevant_cols = ['Path', 'Enlarged Cardiomediastinum', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 
                     'Pneumonia', 'Pneumothorax', 'Pleural Other', 'Support Devices', 'No Finding',
                     'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

    df = df[relevant_cols]
    df = df.replace(np.nan, 0.0)

    df = df.replace(-1.0, 0.0)


    X = df['Path']
    y = df[['Enlarged Cardiomediastinum', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 
            'Pneumonia', 'Pneumothorax', 'Pleural Other', 'Support Devices', 'No Finding',
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']]

    return np.asarray(X), np.asarray(y)


In [48]:
filename = '/content/drive/MyDrive/train.csv'

df = pd.read_csv(filename)
relevant_cols = ['Path', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly' ,
                 'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                 'Pneumonia', 'Atelectasis' ,'Pneumothorax', 'Pleural Effusion',
                 'Pleural Other', 'Fracture', 'Support Devices']

df = df[relevant_cols]
df = df.replace(np.nan, 0.0)

df = df.replace(-1.0, 0.0)


X = df['Path']
y = df[['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly' ,
                 'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                 'Pneumonia', 'Atelectasis' ,'Pneumothorax', 'Pleural Effusion',
                 'Pleural Other', 'Fracture', 'Support Devices']]
y.head()
x1 = np.asarray(X)
y1 = np.asarray(y)



In [49]:
y.head()


,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
print(x1[0])
print(y1[0])

CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [53]:
y.describe()

,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
count,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000,223414.000000
mean,0.100177,0.048332,0.120852,0.472580,0.041116,0.233853,0.066169,0.027031,0.149391,0.087049,0.385773,0.015769,0.040463,0.519220
std,0.300237,0.214467,0.325956,0.499249,0.198560,0.423281,0.248577,0.162173,0.356474,0.281908,0.486778,0.124581,0.197043,0.499632
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [71]:
#(y[1] == 0).sum(axis=1)
#a =y.isin([0]).sum(axis=1)
#print(a)
neg_y = (y[0:18000] == 0).sum(axis=0)
print(neg_y)

No Finding                    15874
Enlarged Cardiomediastinum    17160
Cardiomegaly                  15736
Lung Opacity                  10043
Lung Lesion                   17262
Edema                         14422
Consolidation                 16873
Pneumonia                     17488
Atelectasis                   15558
Pneumothorax                  16334
Pleural Effusion              11667
Pleural Other                 17635
Fracture                      17217
Support Devices                9771
dtype: int64


In [72]:
pos_y = (y[0:18000] == 1).sum(axis=0)
print(pos_y)

No Finding                    2126
Enlarged Cardiomediastinum     840
Cardiomegaly                  2264
Lung Opacity                  7957
Lung Lesion                    738
Edema                         3578
Consolidation                 1127
Pneumonia                      512
Atelectasis                   2442
Pneumothorax                  1666
Pleural Effusion              6333
Pleural Other                  365
Fracture                       783
Support Devices               8229
dtype: int64


In [ ]:
#For example, if a dataset contains 100 positive and 300 negative 
#examples of a single class, then pos_weight for the class should 
#be equal to 300100=3\frac{300}{100}=3100300​=3. 
#The loss would act as if the dataset contains 3×100=3003\times 
#100=3003×100=300 positive examples

#W = Neg/Pos

In [66]:
#weigths
No_Finding  = 201033 / 22381
print('No_Finding',No_Finding)
Enlarged_Cardiomediastinum = 212616 / 10798
print('Enlarged_Cardiomediastinum',Enlarged_Cardiomediastinum)
Cardiomegaly_W = 196414 / 27000
print('Cardiomegaly_W',Cardiomegaly_W)
Lung_Opacity = 117833 / 105581
print('Lung_Opacity',Lung_Opacity)
Lung_Lesion = 214228 / 9186
print('Lung_Lesion',Lung_Lesion)
Edema = 171168 / 52246
print('Edema',Edema)
Consolidation = 208631/14783
print('Consolidation', Consolidation)
Pneumonia = 217375/6039
print('Pneumonia',Pneumonia)
Atelectasis = 190038/33376
print('Atelectasis', Atelectasis)
Pneumothorax = 203966 / 19448
print('Pneumothorax', Pneumothorax)
Pleural_Effusion = 137227 / 86187
print('Pleural_Effusion', Pleural_Effusion)
Pleural_Other = 219891 / 3523
print('Pleural_Othe', Pleural_Other)
Fracture = 214374/9040
print('Fracture', Fracture)
Support_Devices = 107413 / 116001
print('Support_Devices',Support_Devices)

No_Finding 8.982306420624637
Enlarged_Cardiomediastinum 19.690313020929803
Cardiomegaly_W 7.274592592592593
Lung_Opacity 1.1160436063306844
Lung_Lesion 23.321140866536034
Edema 3.2761933927956206
Consolidation 14.112899952648313
Pneumonia 35.99519788044378
Atelectasis 5.693851869606903
Pneumothorax 10.487762237762238
Pleural_Effusion 1.5922006799169248
Pleural_Othe 62.41583877377235
Fracture 23.713938053097344
Support_Devices 0.925966155464177


In [76]:
for i in range(len(pos_y)):
  #print(pos_y[i])
  #print(neg_y[i])
  a = neg_y[i]/pos_y[i]
  print(a)


7.466603951081844
20.428571428571427
6.950530035335689
1.2621591051903984
23.390243902439025
4.0307434320849636
14.971606033717835
34.15625
6.371007371007371
9.804321728691477
1.8422548555187115
48.31506849315068
21.988505747126435
1.1873860736419979
